In [15]:
import numpy as np
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
data = pd.read_csv('/content/netflix_titles.csv')
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick M...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."


In [17]:
data.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [18]:
data = data.dropna(subset=['cast', 'country', 'rating'])

In [19]:
movies = data[data['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri","United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, Timothy Olyphant, Daveed Diggs, Skyler Gisondo, Laura Harrier, Rosalind Chao, Kimberly Quinn, Loretta Devine, Ravi Kapoor",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, Edin Hasanović, Anna Fialová, Marlon Boess, Victor Boccard, Fleur Geffrier, Aziz Dyab, Mélanie Fouché, Elizaveta Maximová","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi, Nassar",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David Spade, Rob Schneider, Salma Hayek, Maria Bello, Maya Rudolph, Colin Quinn, Tim Meadows, Joyce Van Patten",United States,PG-13,Comedies


In [20]:
tv = data[data['type'] == 'TV Show'].reset_index()
tv = tv.drop(['index', 'show_id', 'type', 'date_added', 'release_year', 'duration', 'description'], axis=1)
tv.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick M...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,TV-MA,"International TV Shows, Romantic TV Shows, TV Comedies"
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Hollywood",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton, Antoinette Robertson, John Patrick Amedori, Ashley Blaine Featherson, Marque Richardson, Giancarlo Esposito",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,NaN,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri, Samadhi Zendejas, Eduardo Yáñez, Sonya Smith, Alejandro Camacho, Azela Robinson, Uriel del Toro, Géraldine Bazán, Gabriela Roel, Marcus Ornellas",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV Dramas"


In [21]:
actors = []

for i in movies['cast']:
    actor = re.split(r', \s*', i)
    actors.append(actor)

flat_list = []
for sublist in actors:
    for item in sublist:
        flat_list.append(item)

actors_list = sorted(set(flat_list))

binary_actors = [[0] * 0 for i in range(len(set(flat_list)))]
for i in movies['cast']:
    k = 0
    for j in actors_list:
        if j in i:
            binary_actors[k].append(1.0)
        else:
            binary_actors[k].append(0.0)
        k+=1

binary_actors = pd.DataFrame(binary_actors).transpose()

directors = []
for i in movies['director']:
    if pd.notna(i):
        director = re.split(r', \s*', i)
        directors.append(director)

flat_list2 = []
for sublist in directors:
    for item in sublist:
        flat_list2.append(item)

directors_list = sorted(set(flat_list2))

binary_directors = [[0] * 0 for i in range(len(set(flat_list2)))]
for i in movies['director']:
    k = 0
    for j in directors_list:
        if pd.isna(i):
            binary_directors[k].append(0.0)
        elif j in i:
            binary_directors[k].append(1.0)
        else:
            binary_directors[k].append(0.0)
        k+=1

binary_directors = pd.DataFrame(binary_directors).transpose()

countries = []
for i in movies['country']:
    country = re.split(r', \s*', i)
    countries.append(country)

flat_list3 = []
for sublist in countries:
    for item in sublist:
        flat_list3.append(item)

countries_list = sorted(set(flat_list3))
binary_countries = [[0] * 0 for i in range(len(set(flat_list3)))]

for i in movies['country']:
    k = 0
    for j in countries_list:
        if j in i:
            binary_countries[k].append(1.0)
        else:
            binary_countries[k].append(0.0)
        k+=1

binary_countries = pd.DataFrame(binary_countries).transpose()

genres = []
for i in movies['listed_in']:
    genre = re.split(r', \s*', i)
    genres.append(genre)

flat_list4 = []
for sublist in genres:
    for item in sublist:
        flat_list4.append(item)

genres_list = sorted(set(flat_list4))

binary_genres = [[0] * 0 for i in range(len(set(flat_list4)))]
for i in movies['listed_in']:
    k = 0
    for j in genres_list:
        if j in i:
            binary_genres[k].append(1.0)
        else:
            binary_genres[k].append(0.0)
        k+=1

binary_genres = pd.DataFrame(binary_genres).transpose()
ratings = []

for i in movies['rating']:
    ratings.append(i)

ratings_list = sorted(set(ratings))

binary_ratings = [[0] * 0 for i in range(len(set(ratings_list)))]
for i in movies['rating']:
    k = 0
    for j in ratings_list:
        if j in i:
            binary_ratings[k].append(1.0)
        else:
            binary_ratings[k].append(0.0)
        k+=1

binary_ratings = pd.DataFrame(binary_ratings).transpose()

In [22]:
binary = pd.concat([binary_actors, binary_directors, binary_countries, binary_genres], axis=1,ignore_index=True)
binary

,0,1,2,3,4,5,6,7,8,9,...,29105,29106,29107,29108,29109,29110,29111,29112,29113,29114
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
actors2 = []

for i in tv['cast']:
    actor2 = re.split(r', \s*', i)
    actors2.append(actor2)

flat_list5 = []
for sublist in actors2:
    for item in sublist:
        flat_list5.append(item)

actors_list2 = sorted(set(flat_list5))
binary_actors2 = [[0] * 0 for i in range(len(set(flat_list5)))]

for i in tv['cast']:
    k = 0
    for j in actors_list2:
        if j in i:
            binary_actors2[k].append(1.0)
        else:
            binary_actors2[k].append(0.0)
        k+=1

binary_actors2 = pd.DataFrame(binary_actors2).transpose()
countries2 = []

for i in tv['country']:
    country2 = re.split(r', \s*', i)
    countries2.append(country2)

flat_list6 = []
for sublist in countries2:
    for item in sublist:
        flat_list6.append(item)

countries_list2 = sorted(set(flat_list6))
binary_countries2 = [[0] * 0 for i in range(len(set(flat_list6)))]

for i in tv['country']:
    k = 0
    for j in countries_list2:
        if j in i:
            binary_countries2[k].append(1.0)
        else:
            binary_countries2[k].append(0.0)
        k+=1

binary_countries2 = pd.DataFrame(binary_countries2).transpose()
genres2 = []

for i in tv['listed_in']:
    genre2 = re.split(r', \s*', i)
    genres2.append(genre2)

flat_list7 = []
for sublist in genres2:
    for item in sublist:
        flat_list7.append(item)

genres_list2 = sorted(set(flat_list7))

binary_genres2 = [[0] * 0 for i in range(len(set(flat_list7)))]
for i in tv['listed_in']:
    k = 0
    for j in genres_list2:
        if j in i:
            binary_genres2[k].append(1.0)
        else:
            binary_genres2[k].append(0.0)
        k+=1

binary_genres2 = pd.DataFrame(binary_genres2).transpose()

ratings2 = []

for i in tv['rating']:
    ratings2.append(i)

ratings_list2 = sorted(set(ratings2))

binary_ratings2 = [[0] * 0 for i in range(len(set(ratings_list2)))]
for i in tv['rating']:
    k = 0
    for j in ratings_list2:
        if j in i:
            binary_ratings2[k].append(1.0)
        else:
            binary_ratings2[k].append(0.0)
        k+=1

binary_ratings2 = pd.DataFrame(binary_ratings2).transpose()


In [24]:
binary2 = pd.concat([binary_actors2, binary_countries2, binary_genres2], axis=1, ignore_index=True)
binary2

,0,1,2,3,4,5,6,7,8,9,...,13561,13562,13563,13564,13565,13566,13567,13568,13569,13570
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
def recommender(search):
    cs_list = []
    binary_list = []
    if search in movies['title'].values:
        idx = movies[movies['title'] == search].index.item()
        for i in binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(movies)):
            binary_list2 = []
            for k in binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        movies_copy = movies.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]
        top_results = results.head(5)
        return(top_results)

    elif search in tv['title'].values:
        idx = tv[tv['title'] == search].index.item()
        for i in binary2.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in binary2.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

In [22]:
recommender('The Conjuring')

,title,director,cast,country,rating,listed_in,cos_sim
507,Insidious,James Wan,"Patrick Wilson, Rose Byrne, Lin Shaye, Ty Simp...","United States, Canada",PG-13,"Horror Movies, Thrillers",0.424437
3223,Creep,Patrick Brice,"Mark Duplass, Patrick Brice",United States,R,"Horror Movies, Independent Movies, Thrillers",0.416667
603,The Conjuring 2,James Wan,"Patrick Wilson, Vera Farmiga, Madison Wolfe, F...","Canada, United States, United Kingdom",R,Horror Movies,0.412479
1832,In the Tall Grass,Vincenzo Natali,"Patrick Wilson, Laysla De Oliveira, Avery Whit...","Canada, United States",TV-MA,"Horror Movies, Thrillers",0.400892
3728,Desolation,Sam Patton,"Jaimi Paige, Alyshia Ochse, Toby Nichols, Clau...",United States,TV-MA,"Horror Movies, Thrillers",0.395285


In [23]:
recommender("Child's Play")

,title,director,cast,country,rating,listed_in,cos_sim
3689,Cult of Chucky,Don Mancini,"Fiona Dourif, Michael Therriault, Adam Hurtig,...",United States,R,Horror Movies,0.352941
1461,Wildling,Fritz Böhm,"Bel Powley, Brad Dourif, Liv Tyler, Collin Kel...",United States,R,"Horror Movies, Independent Movies, Sci-Fi & Fa...",0.350070
4768,Stephanie,Akiva Goldsman,"Shree Cooks, Frank Grillo, Anna Torv",United States,R,Horror Movies,0.342997
4434,Nova: Ultimate Mars Challenge,Gail Willumsen,Lance Lewman,United States,TV-G,Movies,0.325396
3160,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,84 min,Movies,0.325396


In [24]:
recommender('Charlie and the Chocolate Factory')

,title,director,cast,country,rating,listed_in,cos_sim
3835,Figaro Pho,Luke Jurevicius,Luke Jurevicius,United Kingdom,TV-Y7,"Children & Family Movies, Comedies",0.394055
4506,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.390095
4802,Sweeney Todd: The Demon Barber of Fleet Street,Tim Burton,"Johnny Depp, Helena Bonham Carter, Alan Rickma...","United States, United Kingdom",R,"Dramas, Horror Movies, Music & Musicals",0.382692
2606,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.361158
1547,A Shaun the Sheep Movie: Farmageddon,"Richard Phelan, Will Becher","Justin Fletcher, John Sparkes, Amalia Vitale, ...","United Kingdom, France, Belgium, Ireland, Unit...",G,"Children & Family Movies, Comedies",0.354005


In [25]:
recommender('Wild Child')

,title,director,cast,country,rating,listed_in,cos_sim
2606,The Kissing Booth,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.440959
4506,Penelope,Mark Palansky,"Christina Ricci, James McAvoy, Catherine O'Har...","United Kingdom, United States",PG,"Children & Family Movies, Comedies, Romantic M...",0.408248
1130,The Kissing Booth 2,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.394405
122,The Kissing Booth 3,Vince Marcello,"Joey King, Joel Courtney, Jacob Elordi, Molly ...","United Kingdom, United States",TV-14,"Comedies, Romantic Movies",0.381881
2501,The Guernsey Literary and Potato Peel Pie Society,Mike Newell,"Lily James, Michiel Huisman, Penelope Wilton, ...","United Kingdom, France, United States",TV-PG,"Dramas, Romantic Movies",0.381881


In [26]:
recommender("Dr. Seuss' The Cat in the Hat")

,title,director,cast,country,rating,listed_in,cos_sim
4419,NOVA: Bird Brain,NaN,Craig Sechler,United States,TV-G,"Children & Family Movies, Documentaries",0.436436
1114,Sugar High,Ariel Boles,Hunter March,United States,TV-G,Children & Family Movies,0.436436
4758,Spy Kids 3: Game Over,Robert Rodriguez,"Daryl Sabara, Sylvester Stallone, Ricardo Mont...",United States,PG,"Children & Family Movies, Comedies",0.422577
2245,Prince of Peoria: A Christmas Moose Miracle,Jon Rosenbaum,"Gavin Lewis, Theodore Barnes, Shelby Simmons, ...",United States,TV-G,"Children & Family Movies, Comedies",0.422577
1363,What a Girl Wants,Dennie Gordon,"Amanda Bynes, Colin Firth, Kelly Preston, Eile...","United States, United Kingdom",PG,"Children & Family Movies, Comedies",0.414039


In [27]:
recommender('Hook')

,title,director,cast,country,rating,listed_in,cos_sim
1918,Sotus The Series,NaN,"Sangpotirat Perawat, Ruangroj Prachaya",Thailand,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",0.405554
843,Little Things,NaN,"Dhruv Sehgal, Mithila Palkar",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",0.405554
1898,Roonpi Secret Love,NaN,"Kanyawee Songmuang, Korapat Kirdpan",Thailand,TV-14,"International TV Shows, Romantic TV Shows, Tee...",0.405554
200,Let's Fight Ghost,Kongkiat Khomsiri,"Suppapong Udomkaewkanjana, Patchanan Jiajirach...",Thailand,TV-14,"International TV Shows, Romantic TV Shows, TV ...",0.381771
1477,Highland: Thailand's Marijuana Awakening,NaN,Sebastian Perry,Thailand,TV-MA,"Docuseries, International TV Shows",0.374634


Recommending TV shows

In [28]:
recommender('After Life')

,title,director,cast,country,rating,listed_in,cos_sim
1699,Extras,NaN,"Ricky Gervais, Stephen Merchant, Ashley Jensen...","United Kingdom, United States",TV-MA,"British TV Shows, TV Comedies",0.564076
1816,Mary Portas: Secret Shopper,NaN,Mary Portas,United Kingdom,TV-14,"British TV Shows, International TV Shows, Real...",0.505076
1636,"Big Dreams, Small Spaces",NaN,Monty Don,United Kingdom,TV-G,"British TV Shows, International TV Shows, Real...",0.505076
1717,Genius of the Ancient World,NaN,Bettany Hughes,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.505076
1718,Genius of the Modern World,NaN,Bettany Hughes,United Kingdom,TV-PG,"British TV Shows, Docuseries, International TV...",0.505076


In [29]:
recommender('Anne with an E')

,title,director,cast,country,rating,listed_in,cos_sim
400,Can You Hear Me?,NaN,"Mélissa Bédard, Ève Landry, Florence Longpré",Canada,TV-MA,"International TV Shows, TV Comedies, TV Dramas",0.507093
621,Restaurants on the Edge,NaN,"Nick Liberato, Karin Bohn, Dennis Prescott",Canada,TV-14,"International TV Shows, Reality TV",0.445435
1739,Heavy Rescue: 401,NaN,Dave Pettitt,Canada,TV-MA,"International TV Shows, Reality TV",0.436436
750,Hip-Hop Evolution,NaN,Shad Kabango,Canada,TV-MA,"Docuseries, International TV Shows",0.436436
1637,Bitten,NaN,"Laura Vandervoort, Greyston Holt, Greg Bryk, S...",Canada,TV-MA,"International TV Shows, TV Dramas, TV Horror",0.428571


In [30]:
recommender('13 Reasons Why')


,title,director,cast,country,rating,listed_in,cos_sim
934,13 Reasons Why: Beyond the Reasons,NaN,"Dylan Minnette, Katherine Langford, Kate Walsh...",United States,TV-MA,"Crime TV Shows, Docuseries",0.453557
1413,Frequency,NaN,"Peyton List, Riley Smith, Mekhi Phifer, Devin ...",United States,TV-14,"Crime TV Shows, TV Dramas, TV Mysteries",0.418330
941,MINDHUNTER,NaN,"Jonathan Groff, Holt McCallany, Anna Torv, Cot...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.404520
281,The Sinner,NaN,"Jessica Biel, Bill Pullman, Christopher Abbott...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.404520
1290,Life Sentence,NaN,Lucy Hale,United States,TV-14,TV Dramas,0.400000


In [31]:
recommender('Breaking Bad')

,title,director,cast,country,rating,listed_in,cos_sim
721,Better Call Saul,NaN,"Bob Odenkirk, Jonathan Banks, Michael McKean, ...",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.554700
143,The Assassination of Gianni Versace,NaN,"Edgar Ramírez, Darren Criss, Ricky Martin, Pen...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Thrillers",0.474342
1721,Get Shorty,NaN,"Ray Romano, Chris O'Dowd",United States,TV-MA,"Crime TV Shows, TV Comedies, TV Dramas",0.441942
1113,Unsolved,NaN,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",0.441942
1882,Prohibition: A Film by Ken Burns and Lynn Novick,NaN,Peter Coyote,United States,TV-PG,"Crime TV Shows, Docuseries",0.408248


In [32]:
recommender('Stranger Things')

,title,director,cast,country,rating,listed_in,cos_sim
1388,Beyond Stranger Things,NaN,"Jim Rash, Matt Duffer, Ross Duffer, Shawn Levy...",United States,TV-14,"Stand-Up Comedy & Talk Shows, TV Mysteries, TV...",0.752618
230,Prank Encounters,NaN,Gaten Matarazzo,United States,TV-MA,"Reality TV, TV Comedies, TV Horror",0.348155
1740,Helix,NaN,"Billy Campbell, Hiroyuki Sanada, Kyra Zagorsky...","United States, Canada",TV-MA,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy",0.301511
997,Scare Tactics,NaN,"Tracy Morgan, Lauren Ash",United States,TV-MA,"Reality TV, TV Comedies, TV Horror",0.301511
45,Brand New Cherry Flavor,NaN,"Rosa Salazar, Eric Lange, Catherine Keener, Je...",United States,TV-MA,"TV Dramas, TV Horror, TV Mysteries",0.295656


Developing Recommendation Engine using Movie/TV show descriptions

In [26]:
movies_des = data[data['type'] == 'Movie'].reset_index()
movies_des = movies_des[['title', 'description']]
movies_des.head()

,title,description
0,Sankofa,"On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past."
1,The Starling,A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward.
2,Je Suis Karl,"After most of her family is murdered in a terrorist bombing, a young woman is unknowingly lured into joining the very group that killed them."
3,Jeans,"When the father of the man she loves insists that his twin sons marry twin sisters, a woman creates an alter ego that might be a bit too convincing."
4,Grown Ups,"Mourning the loss of their beloved junior high basketball coach, five middle-aged pals reunite at a lake house and rediscover the joys of being a kid."


In [27]:
tv_des = data[data['type'] == 'TV Show'].reset_index()
tv_des = tv_des[['title', 'description']]
tv_des.head()

,title,description
0,Blood & Water,"After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
1,Kota Factory,"In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."
2,The Great British Baking Show,"A talented batch of amateur bakers face off in a 10-week competition, whipping up their best dishes in the hopes of being named the U.K.'s best."
3,Dear White People,"Students of color navigate the daily slights and slippery politics of life at an Ivy League college that's not nearly as ""post-racial"" as it thinks."
4,Falsa identidad,Strangers Diego and Isabel flee their home in Mexico and pretend to be a married couple to escape his drug-dealing enemies and her abusive husband.


In [28]:
nltk.download('punkt')
filtered_movies = []
movies_words = []

for text in movies_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    movies_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movies.append(filtered)

movies_words = [val for sublist in movies_words for val in sublist]
movies_words = sorted(set(movies_words))
movies_des['description_filtered'] = filtered_movies
movies_des.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,title,description,description_filtered
0,Sankofa,"On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.","on photo shoot ghana , american model slips back time , enslaved plantation bears witness agony ancestral past ."
1,The Starling,A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward.,a woman adjusting life loss contends feisty bird 's garden — husband 's struggling find forward .
2,Je Suis Karl,"After most of her family is murdered in a terrorist bombing, a young woman is unknowingly lured into joining the very group that killed them.","after family murdered terrorist bombing , young woman unknowingly lured joining group killed ."
3,Jeans,"When the father of the man she loves insists that his twin sons marry twin sisters, a woman creates an alter ego that might be a bit too convincing.","when father loves insists twin marry twin sisters , woman creates alter ego bit convincing ."
4,Grown Ups,"Mourning the loss of their beloved junior high basketball coach, five middle-aged pals reunite at a lake house and rediscover the joys of being a kid.","mourning loss beloved junior high basketball coach , middle-aged pals reunite lake house rediscover joys kid ."


In [29]:
filtered_tv = []
tv_words = []

for text in tv_des['description']:
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    tv_words.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_tv.append(filtered)

tv_words = [val for sublist in tv_words for val in sublist]
tv_words = sorted(set(tv_words))
tv_des['description_filtered'] = filtered_tv
tv_des.head()

,title,description,description_filtered
0,Blood & Water,"After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.","after crossing paths party , cape town sets prove private-school swimming star sister abducted birth ."
1,Kota Factory,"In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life.","in city coaching centers train india ’ finest collegiate minds , earnest unexceptional student friends navigate campus life ."
2,The Great British Baking Show,"A talented batch of amateur bakers face off in a 10-week competition, whipping up their best dishes in the hopes of being named the U.K.'s best.","a talented batch amateur bakers 10-week competition , whipping dishes hopes named u.k. 's ."
3,Dear White People,"Students of color navigate the daily slights and slippery politics of life at an Ivy League college that's not nearly as ""post-racial"" as it thinks.",students color navigate daily slights slippery politics life ivy league college 's `` post-racial '' thinks .
4,Falsa identidad,Strangers Diego and Isabel flee their home in Mexico and pretend to be a married couple to escape his drug-dealing enemies and her abusive husband.,strangers diego isabel flee home mexico pretend married couple escape drug-dealing enemies abusive husband .


In [30]:
movie_word_binary = [[0] * 0 for i in range(len(set(movies_words)))]

for des in movies_des['description_filtered']:
    k = 0
    for word in movies_words:
        if word in des:
            movie_word_binary[k].append(1.0)
        else:
            movie_word_binary[k].append(0.0)
        k+=1

movie_word_binary = pd.DataFrame(movie_word_binary).transpose()

In [31]:
tv_word_binary = [[0] * 0 for i in range(len(set(tv_words)))]

for des in tv_des['description_filtered']:
    k = 0
    for word in tv_words:
        if word in des:
            tv_word_binary[k].append(1.0)
        else:
            tv_word_binary[k].append(0.0)
        k+=1

tv_word_binary = pd.DataFrame(tv_word_binary).transpose()

In [32]:
def recommender2(search):
    cs_list = []
    binary_list = []
    if search in movies_des['title'].values:
        idx = movies_des[movies_des['title'] == search].index.item()
        for i in movie_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(movies_des)):
            binary_list2 = []
            for k in movie_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
            movies_copy = movies_des.copy()
        movies_copy['cos_sim'] = cs_list
        results = movies_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]
        top_results = results.head(5)
        return(top_results)
    elif search in tv_des['title'].values:
        idx = tv_des[tv_des['title'] == search].index.item()
        for i in tv_word_binary.iloc[idx]:
            binary_list.append(i)
        point1 = np.array(binary_list).reshape(1, -1)
        point1 = [val for sublist in point1 for val in sublist]
        for j in range(len(tv)):
            binary_list2 = []
            for k in tv_word_binary.iloc[j]:
                binary_list2.append(k)
            point2 = np.array(binary_list2).reshape(1, -1)
            point2 = [val for sublist in point2 for val in sublist]
            dot_product = np.dot(point1, point2)
            norm_1 = np.linalg.norm(point1)
            norm_2 = np.linalg.norm(point2)
            cos_sim = dot_product / (norm_1 * norm_2)
            cs_list.append(cos_sim)
        tv_copy = tv_des.copy()
        tv_copy['cos_sim'] = cs_list
        results = tv_copy.sort_values('cos_sim', ascending=False)
        results = results[results['title'] != search]
        top_results = results.head(5)
        return(top_results)
    else:
        return("Title not in dataset. Please check spelling.")

Recommending Movies

In [33]:
pd.options.display.max_colwidth = 300
recommender2('The Conjuring')

,title,description,description_filtered,cos_sim
2872,Death Note,"Light Turner finds a supernatural notebook and uses it to mete out death, attracting the attention of a detective, a demon and a girl in his class.","light turner finds supernatural notebook mete death , attracting attention detective , demon girl class .",0.460283
2193,Revenger,"Hell-bent on avenging the murder of his family, a former detective infiltrates a remote island that serves as a prison for vicious death row criminals.","hell-bent avenging murder family , former detective infiltrates remote island serves prison vicious death row criminals .",0.453147
297,A Haunted House 2,"Grieving after a tragedy, Malcolm tries to start fresh with his new girlfriend and her kids — but supernatural shenanigans find their way back to him.","grieving tragedy , malcolm start fresh girlfriend kids — supernatural shenanigans find back .",0.451781
3276,Ég man þig,"Young urbanites renovating a rundown house, and a psychiatrist grieving his son's disappearance, are connected to a supernatural, decades-old secret.","young urbanites renovating rundown house , psychiatrist grieving 's disappearance , connected supernatural , decades-old secret .",0.445435
3900,Ghost House,"While visiting the site of a supernatural shrine in the Thai countryside, a vacationing couple unwittingly upsets a vengeful demon.","while visiting site supernatural shrine thai countryside , vacationing couple unwittingly upsets vengeful demon .",0.434046


In [34]:
recommender2("Child's Play")

,title,description,description_filtered,cos_sim
3927,Good People,A struggling couple can't believe their luck when they find a stash of money in the apartment of a neighbor who was recently murdered.,a struggling couple n't believe luck find stash money apartment neighbor recently murdered .,0.431373
5152,Trip to Bhangarh: Asia's Most Haunted Place,"To amuse themselves, six college friends decide to pay a visit to a fortress believed by some to be the most haunted place in Asia.","to amuse , college friends decide pay visit fortress believed haunted place asia .",0.420084
1358,Material,"A dutiful son must hide his pursuit of stand-up comedy from his staunch father, who expects him to inherit his store and uphold their Muslim beliefs.","a dutiful hide pursuit stand-up comedy staunch father , expects inherit store uphold muslim beliefs .",0.419219
209,Rehmataan,"As unemployment, drug addiction and corruption plague a society, this drama depicts the people who believe there’s still good in the world.","as unemployment , addiction corruption plague society , drama depicts believe ’ world .",0.417483
2009,"Extremely Wicked, Shockingly Evil and Vile",Single mother Liz falls for Ted Bundy and refuses to believe the truth about his crimes for years. A drama based on a true story.,single mother liz falls ted bundy refuses believe truth crimes years . a drama based story .,0.411665


In [35]:
recommender2('Charlie and the Chocolate Factory')

,title,description,description_filtered,cos_sim
5235,Willy Wonka & the Chocolate Factory,Zany Willy Wonka causes a stir when he announces that golden tickets hidden inside his candy bars will admit holders into his secret confectionary.,zany willy wonka stir announces golden tickets hidden inside candy bars admit holders secret confectionary .,0.422200
95,Kyaa Kool Hain Hum 3,"When an unlikely porn actor falls for a woman outside the industry, he employs his co-stars as a stand-in traditional family to impress her father.","when porn actor falls woman industry , employs co-stars stand-in traditional family impress father .",0.372742
1885,Kaake Da Viyah,"In this zany comedy, a man is torn between the girl he loves and the respective women his warring mother and grandmother have chosen for him to marry.","in zany comedy , torn girl loves respective women warring mother grandmother chosen marry .",0.355643
4205,Lechmi,"The spirit of a murdered woman contacts four bachelors, enlisting their help to find her killer.","the spirit murdered woman contacts bachelors , enlisting find killer .",0.353161
3234,BoJack Horseman Christmas Special: Sabrina's Christmas Wish,"It's Christmas, and BoJack wants nothing to do with it. Then Todd shows up with a giant candy cane and an old ""Horsin' Around"" Christmas episode.","it 's christmas , bojack . then todd shows giant candy cane `` horsin ' around '' christmas episode .",0.352034


Recommending TV shows

In [36]:
recommender2('After Life')

,title,description,description_filtered,cos_sim
1150,The Paper,A construction magnate takes over a struggling newspaper and attempts to wield editorial influence for power and personal gain.,a construction magnate takes struggling newspaper attempts wield editorial influence power personal .,0.512182
1377,Longmire,This contemporary crime thriller focuses on a Wyoming sheriff who's rebuilding his life and career following the death of his wife.,this contemporary crime thriller focuses wyoming sheriff 's rebuilding life career death wife .,0.460531
56,Hit & Run,A man searching for the truth behind his wife's death becomes caught up in a dangerous web of secrets and intrigue stretching from New York to Tel Aviv.,a searching truth wife 's death caught dangerous web secrets intrigue stretching new york tel aviv .,0.452509
1719,Gentlemen and Gangsters,"Now on the run, a writer relates his previous year's escapades when he got sucked into the thrilling, sordid orbit of boxer and jazz man Henry Morgan.","now run , writer relates previous year 's escapades sucked thrilling , sordid orbit boxer jazz henry morgan .",0.446211
1276,Welcome to the Family,"When an evicted single mom's estranged father dies, she and his second wife cover up his death after learning they've been written out of his will.","when evicted single mom 's estranged father , wife cover death learning 've written .",0.439024


In [37]:
recommender2('Friends')

,title,description,description_filtered,cos_sim
487,H,"At a dysfunctional hospital in Paris, three bumbling, eccentric medical employees embark on zany misadventures with surgical imprecision.","at dysfunctional hospital paris , bumbling , eccentric medical employees embark zany misadventures surgical imprecision .",0.429377
136,Hap and Leonard,"In late-1980s Texas, two best friends get involved in darkly comic misadventures by hiring themselves out to shady clients for illicit ends.","in late-1980s texas , friends involved darkly comic misadventures hiring shady clients illicit ends .",0.422955
455,Wentworth,Bea Smith is locked up while awaiting trial for the alleged attempted murder of her husband and must learn how life works in prison.,bea smith locked awaiting trial alleged attempted murder husband learn life works prison .,0.404557
355,Juman,"Despite their estranged families, Juman and Hisham fall in love and get married but their different outlooks on life soon threaten their relationship.","despite estranged families , juman hisham fall love married different outlooks life threaten relationship .",0.403846
1513,Crazyhead,Bowling alley worker Amy and nonconformist Raquel kick some serious demon butt while also facing their inner demons as they navigate their early 20s.,bowling alley worker amy nonconformist raquel kick demon butt facing demons navigate early 20s .,0.397215


In [38]:
recommender2('Breaking Bad')

,title,description,description_filtered,cos_sim
662,El Reemplazante,A former high-ranking financial executive finds redemption and romance when he's paroled after a prison sentence and becomes a math teacher.,a former high-ranking financial executive finds redemption romance 's paroled prison sentence math teacher .,0.474252
356,Mr. Iglesias,Hilarious high school teacher Gabriel Iglesias tries to make a difference in the lives of some smart but underperforming students at his alma mater.,hilarious high school teacher gabriel iglesias make difference lives smart underperforming students alma mater .,0.467099
1139,Marvel's The Punisher,A former Marine out to punish the criminals responsible for his family's murder finds himself ensnared in a military conspiracy.,a former marine punish criminals responsible family 's murder finds ensnared military conspiracy .,0.456435
1908,Servant of the People,"After a Ukrainian high school teacher's tirade against government corruption goes viral on social media, he finds himself the country's new president.","after ukrainian high school teacher 's tirade government corruption viral social media , finds country 's president .",0.446910
570,Alexa & Katie,"Alexa is battling cancer. But with her best friend, Katie, by her side, she's also starting high school – and ready for whatever comes next.","alexa battling cancer . but friend , katie , side , 's starting high school – ready .",0.427121
